In [1]:
# Black–Litterman + Markowitz (todo en frecuencia anual)

import os
import numpy as np
import pandas as pd
from numpy.linalg import eigvalsh
from scipy.optimize import minimize, Bounds, LinearConstraint

# --- Configuración ---
FILE_XLSX = '/Users/george/Downloads/PRECIO 10 ACT DESDE 2023.xlsx'
FONDOS = ['Renta UF','Renta internacional','Acciones Chile','Acciones Mundo','HACK','TLH','IGSB','IEF','SPY','IWO']
R_TARGET = 0.10
LAMBDA = 4.0
RF = 0.0
LB, UB = 0.0, 1.0
IDX_VISTAS = [0,1,2,3,5,6,7,8,9]
ETAS = [1.0,1.0,1.0,1.0,-1.0,-2.0,2.0,-2.0,1.0]
C_CONF = 1.0
U_BY_VIEW = None
TAU = 0.25
COV_MODE_TAU = 'predictive'
EW_LAMBDA = 0.94
EW_DEMEAN = True
USE_EW_MEAN = False
W_REF = None

# --- Datos ---

def load_prices(path, fondos, date_hint='Fecha'):
    ext = str(path).lower().rsplit('.', 1)[-1] if isinstance(path, str) else ''
    if ext == 'xlsx':
        try:
            import openpyxl  # type: ignore
            df = pd.read_excel(path, engine='openpyxl')
        except Exception as e:
            alt = os.path.splitext(path)[0] + '.csv'
            if os.path.exists(alt):
                df = pd.read_csv(alt)
            else:
                raise ImportError('Falta openpyxl o un CSV alternativo con el mismo nombre.') from e
    elif ext == 'csv':
        df = pd.read_csv(path)
    else:
        try:
            df = pd.read_excel(path)
        except Exception:
            df = pd.read_csv(path)
    date_cols = [c for c in df.columns if str(c).lower().startswith(date_hint.lower())] or [c for c in df.columns if np.issubdtype(df[c].dtype, np.datetime64)]
    if not date_cols:
        raise ValueError('No se encontró columna de fechas.')
    cdate = date_cols[0]
    df[cdate] = pd.to_datetime(df[cdate], dayfirst=True, errors='coerce')
    df = df.dropna(subset=[cdate]).sort_values(cdate).set_index(cdate)
    cols_present = [c for c in fondos if c in df.columns]
    if len(cols_present) == len(fondos):
        prices = df[cols_present].astype(float)
    else:
        num = df.select_dtypes(include=[np.number])
        if num.shape[1] < len(fondos):
            raise ValueError('No hay suficientes columnas numéricas para los activos.')
        prices = num.iloc[:, :len(fondos)].copy(); prices.columns = fondos
    return prices

def compute_returns(prices):
    return prices.pct_change().replace([np.inf,-np.inf],np.nan).dropna(how='any')

# --- Utilidades numéricas ---

def symmetrize(S): return 0.5*(S+S.T)

def make_psd(S, eps=None):
    S = symmetrize(np.asarray(S,float))
    lam_min = float(eigvalsh(S).min())
    if lam_min >= 0: return S
    if eps is None:
        scale = abs(np.trace(S))/S.shape[0] if np.isfinite(np.trace(S)) else 1.0
        eps = max(1e-8,1e-6*scale)
    return S + (-(lam_min)+eps)*np.eye(S.shape[0])

def spd_with_jitter(A, eps=1e-10):
    A = symmetrize(np.asarray(A,float))
    lam_min = float(eigvalsh(A).min())
    if lam_min < eps: A = A + (eps - lam_min)*np.eye(A.shape[0])
    return A

# --- Proyección presupuesto/cotas ---

def project_to_capped_simplex(w, lb, ub, s=1.0, tol=1e-12, max_iter=100):
    w = np.asarray(w,float); n = w.size
    lbv = np.full(n,lb) if np.isscalar(lb) else np.asarray(lb,float)
    ubv = np.full(n,ub) if np.isscalar(ub) else np.asarray(ub,float)
    lo = float((w-ubv).min()); hi = float((w-lbv).max())
    for _ in range(max_iter):
        tau = 0.5*(lo+hi)
        w_tau = np.clip(w - tau, lbv, ubv)
        g = w_tau.sum()-s
        if abs(g) <= tol: break
        if g>0: lo = tau
        else:   hi = tau
    return w_tau

def enforce_budget(w, lb, ub, name='', s=1.0, tol=1e-10, verbose=True):
    w = np.asarray(w,float)
    s0 = float(w.sum()); lbv = np.full(w.size,lb) if np.isscalar(lb) else np.asarray(lb,float); ubv = np.full(w.size,ub) if np.isscalar(ub) else np.asarray(ub,float)
    if (abs(s0-s)>tol) or (np.min(w-lbv)<0) or (np.max(w-ubv)>0):
        w = project_to_capped_simplex(w, lb, ub, s)
        if verbose: print(f"[fix] {name}: sum -> {w.sum():.12f}")
    return w

# --- EWMA ---

def ewma_mean(df, lam=0.94):
    if not (0.0<lam<1.0): raise ValueError('lam fuera de (0,1)')
    alpha = 1.0 - lam
    return df.ewm(alpha=alpha, adjust=False).mean().iloc[-1]

def ewma_covariance(df, lam=0.94, demean=True):
    if not (0.0<lam<1.0): raise ValueError('lam fuera de (0,1)')
    X = np.asarray(df.to_numpy(), float); T, n = X.shape; S = np.zeros((n,n), float)
    if demean:
        m = X[0].copy()
        for t in range(1,T):
            x = X[t]; m_prev = m; m = lam*m_prev + (1.0-lam)*x
            xc = x - m_prev; S = lam*S + (1.0-lam)*np.outer(xc, xc)
    else:
        for t in range(T):
            x = X[t]; S = lam*S + (1.0-lam)*np.outer(x, x)
    return make_psd(S)

# --- Conversión a anual ---

def daily_to_annual(mu_d, Sigma_d):
    mu_a = (1.0 + np.asarray(mu_d))**252 - 1.0
    Sigma_a = np.asarray(Sigma_d)*252.0
    return mu_a, Sigma_a

# --- Optimizaciones ---

def gmv_portfolio(Sigma, lb, ub):
    Sigma = make_psd(Sigma); n = Sigma.shape[0]
    bds = Bounds(lb, ub); w0 = np.clip(np.full(n,1.0/n), lb, ub); w0 += (1.0 - w0.sum())/n
    cons = [LinearConstraint(np.ones((1,n)),1.0,1.0)]
    res = minimize(lambda w: 0.5*w@Sigma@w, w0, method='trust-constr', jac=lambda w: Sigma@w, hess=lambda _w: Sigma, constraints=cons, bounds=bds, options={'gtol':1e-10,'xtol':1e-12,'maxiter':2000,'verbose':0})
    if not res.success: raise RuntimeError(f"GMV no convergió: {res.message}")
    w = enforce_budget(res.x, lb, ub, name='GMV'); var = float(w@Sigma@w)
    return w, var

def minvar_with_target(mu, Sigma, R_target, lb, ub):
    mu = np.asarray(mu).ravel(); Sigma = make_psd(Sigma); n = mu.size
    bds = Bounds(lb, ub); w0 = np.clip(np.full(n,1.0/n), lb, ub); w0 += (1.0 - w0.sum())/n
    cons = [LinearConstraint(np.ones((1,n)),1.0,1.0), LinearConstraint(mu.reshape(1,-1), R_target, R_target)]
    res = minimize(lambda w: 0.5*w@Sigma@w, w0, method='trust-constr', jac=lambda w: Sigma@w, hess=lambda _w: Sigma, constraints=cons, bounds=bds, options={'gtol':1e-10,'xtol':1e-12,'maxiter':2000,'verbose':0})
    if not res.success: raise RuntimeError(f"MinVar target no convergió: {res.message}")
    w = enforce_budget(res.x, lb, ub, name='MinVar target'); ret = float(w@mu); var = float(w@Sigma@w)
    return w, ret, var

def max_utility(mu, Sigma, risk_aversion, lb, ub):
    mu = np.asarray(mu).ravel(); Sigma = make_psd(Sigma); n = mu.size
    bds = Bounds(lb, ub); w0 = np.clip(np.full(n,1.0/n), lb, ub); w0 += (1.0 - w0.sum())/n
    cons = [LinearConstraint(np.ones((1,n)),1.0,1.0)]
    def negU(w): return -(w@mu - risk_aversion*(w@Sigma@w))
    def g(w):    return -(mu - 2.0*risk_aversion*(Sigma@w))
    def H(_w):   return  2.0*risk_aversion*Sigma
    res = minimize(negU, w0, method='trust-constr', jac=g, hess=H, constraints=cons, bounds=bds, options={'gtol':1e-10,'xtol':1e-12,'maxiter':2000,'verbose':0})
    if not res.success: raise RuntimeError(f"Utilidad no convergió: {res.message}")
    w = enforce_budget(res.x, lb, ub, name='MaxUtil'); ret = float(w@mu); var = float(w@Sigma@w)
    return w, ret, var

# --- Reporte ---

def optimizar_y_reportar_bl(mu, Sigma, etiqueta, risk_aversion=LAMBDA, lb=LB, ub=UB, R_target=None):
    mu = np.asarray(mu).ravel(); Sigma = make_psd(Sigma); n = mu.size
    lbv, ubv = np.full(n,lb), np.full(n,ub)
    w_gmv, v_gmv = gmv_portfolio(Sigma, lbv, ubv); r_gmv = float(w_gmv@mu)
    w_utl, r_utl, v_utl = max_utility(mu, Sigma, risk_aversion, lbv, ubv)
    have_target = R_target is not None
    if have_target:
        R_t = np.clip(R_target, float(mu.min()), float(mu.max()))
        w_min, r_min, v_min = minvar_with_target(mu, Sigma, R_t, lbv, ubv)
    print(f"================= {etiqueta} =================")
    print("[1] GMV — mínima varianza (sin objetivo)")
    print("Pesos:", np.round(w_gmv,4)); print(f"Suma pesos: {w_gmv.sum():.6f}")
    print(f"Ret anual: {r_gmv:.4%} | Vol anual: {np.sqrt(v_gmv):.4%}")
    print("[2] Máx Utilidad media–varianza")
    print(f"λ={risk_aversion:.4f}"); print("Pesos:", np.round(w_utl,4)); print(f"Suma pesos: {w_utl.sum():.6f}")
    print(f"Ret anual: {r_utl:.4%} | Vol anual: {np.sqrt(v_utl):.4%}")
    if have_target:
        print("[3] Min Var con retorno objetivo (anclado)")
        print("Pesos:", np.round(w_min,4)); print(f"Suma pesos: {w_min.sum():.6f}")
        print(f"Ret anual: {r_min:.4%} | Vol anual: {np.sqrt(v_min):.4%} | R*_obj: {R_t:.4%}")
    out = {'w_gmv':w_gmv,'ret_gmv':r_gmv,'var_gmv':v_gmv,'w_util':w_utl,'ret_util':r_utl,'var_util':v_utl}
    if have_target: out.update({'w_minvar':w_min,'ret_minvar':r_min,'var_minvar':v_min})
    return out

def optimizar_y_reportar(mu, Sigma, etiqueta, R_target, risk_aversion, lb=LB, ub=UB):
    mu = np.asarray(mu).ravel(); Sigma = make_psd(Sigma); n = mu.size
    lbv, ubv = np.full(n,lb), np.full(n,ub)
    R_t = np.clip(R_target, float(mu.min()), float(mu.max()))
    w_min, r_min, v_min = minvar_with_target(mu, Sigma, R_t, lbv, ubv)
    w_utl, r_utl, v_utl = max_utility(mu, Sigma, risk_aversion, lbv, ubv)
    print(f"================= {etiqueta} =================")
    print("[1] Min Var con retorno objetivo"); print("Pesos:", np.round(w_min,4)); print(f"Suma pesos: {w_min.sum():.6f}")
    print(f"Ret anual: {r_min:.4%} | Vol anual: {np.sqrt(v_min):.4%} | R*_obj: {R_t:.4%}")
    print("[2] Máx Utilidad media–varianza"); print(f"λ={risk_aversion:.4f}"); print("Pesos:", np.round(w_utl,4)); print(f"Suma pesos: {w_utl.sum():.6f}")
    print(f"Ret anual: {r_utl:.4%} | Vol anual: {np.sqrt(v_utl):.4%}")
    return {'w_minvar':w_min,'ret_minvar':r_min,'var_minvar':v_min,'w_util':w_utl,'ret_util':r_utl,'var_util':v_utl}

# --- Reverse optimization (π) y views ---

def calibrate_delta_and_pi(mu, Sigma, w_ref, rf=0.0):
    mu_mkt = float(w_ref@mu); var_mkt = float(w_ref@Sigma@w_ref)
    if var_mkt <= 0: raise ValueError('Varianza del mercado no positiva.')
    delta = (mu_mkt - rf)/var_mkt; pi = delta*(Sigma@w_ref)
    return delta, pi

def build_views_eta_sigma(pi, Sigma, idx_views, etas, c_conf=1.0, u_by_view=None, n_assets=None):
    pi = np.asarray(pi).ravel(); Sigma = make_psd(Sigma); n = pi.size if n_assets is None else n_assets
    idx_views = list(idx_views); etas = np.asarray(etas,float).ravel()
    if len(idx_views) != len(etas): raise ValueError('Dimensión de views incompatible.')
    K = len(idx_views); P = np.zeros((K,n))
    for k,j in enumerate(idx_views): P[k,j] = 1.0
    PSPT = symmetrize(P@Sigma@P.T); std_views = np.sqrt(np.clip(np.diag(PSPT),0.0,None))
    base = P@pi; v = base + etas*std_views
    if u_by_view is None: D = np.eye(K)
    else:
        u = np.asarray(u_by_view,float).ravel();
        if u.size != K: raise ValueError('Dimensión de u_by_view incompatible.')
        D = np.diag(u)
    Omega = spd_with_jitter((1.0/c_conf)*(D@PSPT@D), eps=1e-10)
    return P, v, Omega

# --- Black–Litterman ---

def black_litterman_market(pi, Sigma, P, v, Omega):
    K = spd_with_jitter(P@Sigma@P.T + Omega)
    delta = np.linalg.solve(K, (v - P@pi))
    mu_bl = pi + Sigma@P.T@delta
    Sigma_bl = Sigma - Sigma@P.T@np.linalg.solve(K, P@Sigma)
    return mu_bl, make_psd(Sigma_bl)

def black_litterman_tau(pi, Sigma, P, v, Omega, tau, cov_mode='predictive'):
    K = spd_with_jitter(tau*(P@Sigma@P.T) + Omega)
    delta = np.linalg.solve(K, (v - P@pi))
    mu_blt = pi + tau*(Sigma@P.T@delta)
    if cov_mode == 'same':
        Sigma_blt = Sigma.copy()
    elif cov_mode == 'predictive':
        post_mu_cov = tau*Sigma - tau*Sigma@P.T@np.linalg.solve(K, P@(tau*Sigma))
        Sigma_blt = make_psd(Sigma + post_mu_cov)
    else:
        raise ValueError('cov_mode inválido')
    return mu_blt, Sigma_blt

# --- Checks ---

def bl_sanity_checks(asset_names, P, pi, Sigma, mu_bl, mu_blt, v):
    print("--- Check mapping de P (view -> activos) ---")
    for k in range(P.shape[0]):
        cols = np.where(np.abs(P[k])>1e-12)[0].tolist()
        print(f"View {k+1}: cols={cols} -> {[asset_names[j] for j in cols]} ; coef={P[k, cols]}")
    resid_mkt = v - P@mu_bl; resid_tau = v - P@mu_blt
    print("||v - P μ_BL (market-based)|| =", float(np.linalg.norm(resid_mkt)))
    print("||v - P μ_BLτ (clásico τ)||  =", float(np.linalg.norm(resid_tau)))
    d_mkt = mu_bl - pi; d_tau = mu_blt - pi
    print("Top |Δμ| (market-based):", np.argsort(-np.abs(d_mkt))[:5].tolist())
    print("Top |Δμ| (τ):", np.argsort(-np.abs(d_tau))[:5].tolist())

# --- Ejecución ---
prices = load_prices(FILE_XLSX, FONDOS)
rets_d = compute_returns(prices)
mu_d = ewma_mean(rets_d, lam=EW_LAMBDA).to_numpy() if USE_EW_MEAN else rets_d.mean().to_numpy()
Sigma_d = ewma_covariance(rets_d, lam=EW_LAMBDA, demean=EW_DEMEAN)
mu, Sigma = daily_to_annual(mu_d, Sigma_d)
asset_names = list(rets_d.columns); n = len(asset_names)

res_clasico = optimizar_y_reportar(
    mu=mu, Sigma=Sigma, etiqueta='CLÁSICO (μ, Σ anuales)',
    R_target=R_TARGET, risk_aversion=LAMBDA, lb=LB, ub=UB
)

w_ref_min = res_clasico['w_minvar']
w_ref_utl = res_clasico['w_util']
_, pi_min  = calibrate_delta_and_pi(mu, Sigma, w_ref_min, rf=RF)
_, pi_utl  = calibrate_delta_and_pi(mu, Sigma, w_ref_utl, rf=RF)
Sigma_prior = make_psd(Sigma)

P_min, v_min, Omega_min = build_views_eta_sigma(pi_min, Sigma_prior, IDX_VISTAS, ETAS, c_conf=C_CONF, u_by_view=U_BY_VIEW, n_assets=n)
P_utl, v_utl, Omega_utl = build_views_eta_sigma(pi_utl, Sigma_prior, IDX_VISTAS, ETAS, c_conf=C_CONF, u_by_view=U_BY_VIEW, n_assets=n)

mu_bl_min,  Sigma_bl_min  = black_litterman_market(pi_min, Sigma_prior, P_min, v_min, Omega_min)
mu_blt_min, Sigma_blt_min = black_litterman_tau    (pi_min, Sigma_prior, P_min, v_min, Omega_min, TAU, cov_mode=COV_MODE_TAU)
mu_bl_utl,  Sigma_bl_utl  = black_litterman_market(pi_utl, Sigma_prior, P_utl, v_utl, Omega_utl)
mu_blt_utl, Sigma_blt_utl = black_litterman_tau    (pi_utl, Sigma_prior, P_utl, v_utl, Omega_utl, TAU, cov_mode=COV_MODE_TAU)

print("=== Flujo MINVAR: Black–Litterman (Market-based) ===")
print('μ_BL anual:  ', np.round(mu_bl_min, 8))
print(f"Σ_BL anual  (traza={np.trace(Sigma_bl_min):.6e}, min_eig={eigvalsh(Sigma_bl_min).min():.6e})")
print("=== Flujo MINVAR: Black–Litterman (Clásico, τ) ===")
print(f"τ utilizado: {TAU}")
print('μ_BLτ anual: ', np.round(mu_blt_min, 8))
print(f"Σ_BLτ anual (traza={np.trace(Sigma_blt_min):.6e}, min_eig={eigvalsh(Sigma_blt_min).min():.6e}, modo='{COV_MODE_TAU}')")

print("=== Flujo UTILIDAD: Black–Litterman (Market-based) ===")
print('μ_BL anual:  ', np.round(mu_bl_utl, 8))
print(f"Σ_BL anual  (traza={np.trace(Sigma_bl_utl):.6e}, min_eig={eigvalsh(Sigma_bl_utl).min():.6e})")
print("=== Flujo UTILIDAD: Black–Litterman (Clásico, τ) ===")
print(f"τ utilizado: {TAU}")
print('μ_BLτ anual: ', np.round(mu_blt_utl, 8))
print(f"Σ_BLτ anual (traza={np.trace(Sigma_blt_utl):.6e}, min_eig={eigvalsh(Sigma_blt_utl).min():.6e}, modo='{COV_MODE_TAU}')")

res_prior_min  = optimizar_y_reportar_bl(pi_min,   Sigma_prior, 'PRIOR (π anclado en MinVar)', risk_aversion=LAMBDA, lb=LB, ub=UB)
res_bl_mkt_min = optimizar_y_reportar_bl(mu_bl_min,  Sigma_bl_min,  'BL Market-based (anchor MinVar)', risk_aversion=LAMBDA, lb=LB, ub=UB)
res_bl_tau_min = optimizar_y_reportar_bl(mu_blt_min, Sigma_blt_min, 'BL Clásico con τ (anchor MinVar)', risk_aversion=LAMBDA, lb=LB, ub=UB)

res_prior_utl  = optimizar_y_reportar_bl(pi_utl,   Sigma_prior, 'PRIOR (π anclado en MaxUtil)', risk_aversion=LAMBDA, lb=LB, ub=UB)
res_bl_mkt_utl = optimizar_y_reportar_bl(mu_bl_utl,  Sigma_bl_utl,  'BL Market-based (anchor MaxUtil)', risk_aversion=LAMBDA, lb=LB, ub=UB)
res_bl_tau_utl = optimizar_y_reportar_bl(mu_blt_utl, Sigma_blt_utl, 'BL Clásico con τ (anchor MaxUtil)', risk_aversion=LAMBDA, lb=LB, ub=UB)

bl_sanity_checks(asset_names, P_min, pi_min, Sigma_prior, mu_bl_min, mu_blt_min, v_min)
bl_sanity_checks(asset_names, P_utl, pi_utl, Sigma_prior, mu_bl_utl, mu_blt_utl, v_utl)

for r in (res_prior_min,res_bl_mkt_min,res_bl_tau_min,res_prior_utl,res_bl_mkt_utl,res_bl_tau_utl):
    assert abs(r['w_gmv'].sum()-1.0) < 1e-8
    assert abs(r['w_util'].sum()-1.0) < 1e-8

================= CLÁSICO (μ, Σ anuales) =================
[1] Min Var con retorno objetivo
Pesos: [0.5812 0.     0.0094 0.     0.0458 0.     0.     0.     0.3636 0.    ]
Suma pesos: 1.000000
Ret anual: 10.0000% | Vol anual: 4.2884% | R*_obj: 10.0000%
[2] Máx Utilidad media–varianza
λ=4.0000
Pesos: [1.000e-04 0.000e+00 1.000e-04 1.000e-04 1.295e-01 0.000e+00 0.000e+00
 0.000e+00 8.702e-01 0.000e+00]
Suma pesos: 1.000000
Ret anual: 16.0176% | Vol anual: 10.2327%
=== Flujo MINVAR: Black–Litterman (Market-based) ===
μ_BL anual:   [ 0.00991276 -0.000703    0.14068195  0.22949521  0.2785531  -0.01024443
 -0.01024443  0.02102157  0.12649243  0.33575436]
Σ_BL anual  (traza=5.107519e-02, min_eig=4.743385e-20)
=== Flujo MINVAR: Black–Litterman (Clásico, τ) ===
τ utilizado: 0.25
μ_BLτ anual:  [ 0.00718199 -0.00524458  0.10888158  0.20378084  0.30257473 -0.00343331
 -0.00343331 -0.00590183  0.18505879  0.29057797]
Σ_BLτ anual (traza=1.101976e-01, min_eig=1.101975e-08, modo='predictive')
=== Flujo